# AI Agents

In [ ]:
pip install -q langgraph requests rich

## Large Language Model 

In [ ]:
from langchain_aws import ChatBedrock
from langchain_aws import ChatBedrock
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()


llm = ChatOpenAI(
    model="meta-llama/Llama-3.1-8B-Instruct",  
    openai_api_base=f"http://{os.getenv('MODEL_IP')}:8000/v1",
    openai_api_key="EMPTY"  # vLLM doesn’t require real keys
)

# llm = ChatBedrock(
#     model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
#     model_kwargs=dict(temperature=0),
#     region="us-east-1"
# )

##  Tools

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
BASE_URL=os.getenv("BASE_URL")
USERNAME=os.getenv("USERNAME")
PASSWORD=os.getenv("PASSWORD")

In [ ]:
# Authenticate

import requests
import warnings
import urllib3

warnings.simplefilter('ignore', urllib3.exceptions.InsecureRequestWarning)

url = f"{BASE_URL}/dna/system/api/v1/auth/token"
response = requests.post(url, auth=(USERNAME, PASSWORD), verify=False)

if "error" in response.text:
    raise ValueError("ERROR: Failed to retrieve Access Token! REASON: {}".format(response.json()["error"]))
else:
    global dna_token
    dna_token = response.json()["Token"]



In [ ]:
dna_token

In [ ]:
from langchain.tools import tool
import csv
from io import StringIO

@tool("get_devices")
def get_devices():
    """
    Retrieves the list of devices in the DNA Center


    Returns:
    - list: A list of dictionary with the Device. Including type, hostname, software, version etc
    """
    url = f"{BASE_URL}/dna/intent/api/v1/network-device"
    headers = {"X-Auth-Token": dna_token}
    response = requests.get(url, headers=headers, verify=False)
    response.raise_for_status()
    return response.json()['response']

@tool("save_csv")
def save_csv(csv_string: str, filename: str):
    """
    Saves a CSV string to a file.

    Parameters:
    - csv_string (str):  CSV content as a string. Input is a text separates by commas (,). Line changr per row
    - filename (str): Filename to save (should end with .csv)

    Returns:
    - status: Informs if the operation was sucessful or not
    """
    # Use StringIO to treat string as a file
    f = StringIO(csv_string)
    reader = csv.reader(f)
    
    # Write to actual CSV file
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        for row in reader:
            writer.writerow(row)
    return f"File {filename} saved" 

## Agent

In [ ]:
from langgraph.prebuilt import create_react_agent

system_prompt = """
You are a Cisco DNA Center expert AI.

Rules:
1. Only call ONE tool per response, if a tool is actually needed to answer the user.
2. If the user’s question can be answered without a tool, answer normally in text.
"""

dna_center_agent = create_react_agent(
    llm,
    [get_devices, save_csv],
    prompt=system_prompt
)

In [ ]:
dna_center_agent

In [ ]:
response = dna_center_agent.invoke({"messages": "Hi!"})
print(response['messages'][-1].content)

In [ ]:
response = dna_center_agent.invoke({"messages": "What devices do I have configured?"})
print(response['messages'][-1].content)

In [ ]:
import rich
from langchain_core.messages import HumanMessage

initial_message = HumanMessage(
    content="Generate a CSV file with the list of devices. List the hostname, serial number and SW version. Use a single tool per iteration!"
)


for event in dna_center_agent.stream(
    {"messages": [initial_message]},
    stream_mode="updates",
):
    for _, message_or_messages in event.items():
        if isinstance(message_or_messages["messages"], list):
            for message in message_or_messages["messages"]:
                rich.print(message.pretty_repr())
        else:
            rich.print(message_or_messages["messages"].pretty_repr())